## Create Myanmar Maps with cluster annotations using voronyi diagrams
### Author: T. Janus
### December, 2023

In [ ]:
library(tidyverse)
library(dplyr)
library(sf)
library(Hmisc)
library(ggplot2)
library(colorspace)
library(ggspatial)
library(readxl)
library(latex2exp)
library(tmap)
library(ggtext)
#library(cowplot)
library(ggvoronoi)
library(grid)
library(ggrepel)

In [ ]:
# Define Workbook constants
mya_epsg_code <- "EPSG:32646"
webcrs <- "EPSG:3857"
googlecrs <- "EPSG:4326"
# Ust the same color map as implemented in the Python Jupyter notebook with cluster plots
cluster_colors = c(
    "#8DD3C7", 
    "#FFFFB3",
    "#BEBADA", 
    "#80B1D3",
    "#FDB462",
    "#B3DE69")
plot_map_with_elevation <- FALSE

In [ ]:
## Add raster libraries
library(raster)
raster_libs <-c("rayvista", "elevatr", "rayshader", "sf", "giscoR")
invisible(lapply(raster_libs, library, character.only = TRUE))
# Get the myanmar outline with 
get_mya_sf <- function(){
    mya_sf <- gisco_get_countries(verbose = T,
        #epsg = "4326",
        country="MM",
        #resolution = '10'
    ) |> 
    sf::st_transform(crs = mya_epsg_code)
    return(mya_sf)
}
mya_sf <- get_mya_sf()

if (plot_map_with_elevation == TRUE)
{
    print("Getting elevation raster")
    get_mya_dem <- function(){
        mya_dem <- elevatr::get_elev_raster(
            locations = mya_sf,
            z = 5,
            clip="locations")
        return(mya_dem)
    }
    mya_dem <- get_mya_dem()
    names(mya_dem) <- "elevation"
    mya_dem.df <- as.data.frame(mya_dem, xy=TRUE)
    mya_dem.df <- mya_dem.df %>% drop_na(elevation)
}

## Read in a myanmar contour shape file and transform the crs

In [ ]:
# Load the cluster data saved in the Python notebook (Notebook_8)
# Use bgmm derived clusters
feat_clusters_ch4_path <- file.path(
    "intermediate", "density_mapping", "ch4", "bgmm_clusters_feat_ch4.xlsx")
feat_clusters_co2_path <- file.path(
    "intermediate", "density_mapping", "co2", "bgmm_clusters_feat_co2.xlsx")

mya_outline_shp_path <- file.path(
    "bin", "gis_layers", "myanmar_outline", "Myanmar_outline.shp")
mya_neigbours_shp_path <- file.path(
    "bin", "gis_layers", "myanmar_outline", "around_mya", "myanmar_surrounding.shp")

bgmm_feat_ch4 <- readxl::read_excel(feat_clusters_ch4_path)
bgmm_feat_ch4$cluster <- as.integer(bgmm_feat_ch4$cluster)
bgmm_feat_co2 <- readxl::read_excel(feat_clusters_co2_path)
bgmm_feat_co2$cluster <- as.integer(bgmm_feat_co2$cluster)
# Read Myanmar outine shape
mya_outline <- sf::st_read(
    dsn = mya_outline_shp_path, quiet = TRUE, geometry_column = 'geometry') %>% 
    st_transform(mya_epsg_code)
# Read a shape file which is a shape of a bounding box and used for masking the voronoi graph
# that extends beyond the contours of Myanmar
mya_outside <- sf::st_read(
    dsn = mya_neigbours_shp_path, quiet = TRUE, geometry_column = 'geometry') %>% 
    st_transform(mya_epsg_code)

# Create a bounding box around myanmar
df_sf <- sf::st_as_sf(mya_outline, coords=c("Lon", "Lat"), crs=googlecrs, agr = "identity")
df_xy <- sf::st_transform(df_sf, crs=mya_epsg_code)
bbox <- sf::st_bbox(df_xy)

expansion_factor <- 0.1
# Expand box by 20% to give a little extra room
Dx <- (bbox[["xmax"]]-bbox[["xmin"]])*expansion_factor/2
Dy <- (bbox[["ymax"]]-bbox[["ymin"]])*expansion_factor/2
bbox["xmin"] <- bbox["xmin"] - Dx
bbox["xmax"] <- bbox["xmax"] + Dx
bbox["ymin"] <- bbox["ymin"] - Dy
bbox["ymax"] <- bbox["ymax"] + Dy

# Create a bounding box
bb <- c(bbox["xmin"], bbox["ymin"], bbox["xmax"], bbox["ymax"])

## Load other spatial data for making a map and include information about clusters

In [ ]:
# Shape file of reservoirs with attached emissions and ifc data
neighbours_shp_path <- file.path("bin", "gis_layers", "World_Countries_Generalized.shp")
mya_rivers_shp_path <- file.path("bin", "gis_layers", "mya_rivers.shp")
res_with_emisisons_shp_path <- file.path("bin", "heet_outputs_MIN_LOW_PRI", "reservoirs_updated.shp")

mya_reservoirs <- sf::st_read(dsn = res_with_emisisons_shp_path, quiet = TRUE) %>% 
    st_transform(mya_epsg_code) %>% 
    filter(!is.na(tot_em)) %>% 
    mutate(type = replace(type, type == "unknown", "hydroelectric")) %>%
    rename(reservoir_type = type)
# Centroids / center points of reservoirs
res_centroids <- sf::st_centroid(
    mya_reservoirs, of_largest_polygon = TRUE, agr="constant") %>% 
    st_transform(mya_epsg_code) %>%
    arrange(desc(r_area_km2)) %>%
    distinct() %>%
    dplyr::mutate(center_lon = sf::st_coordinates(.)[,1],
                  center_lat = sf::st_coordinates(.)[,2])
# Largest rivers in Myanmar (for allowing better indication of where reservoirs are positioned)
mya_rivers <- sf::st_read(dsn = mya_rivers_shp_path, quiet = TRUE)  %>% 
    st_transform(mya_epsg_code)
# Neighbouring countries
mya_neighbours <- sf::st_read(dsn = neighbours_shp_path, quiet = TRUE)  %>% 
    st_transform(mya_epsg_code) %>%
    filter(COUNTRY != "Myanmar")
# Crop the neighbouring countries layer to the bounding box such that the layer does not go over the border
mya_neighbours_crop <- st_crop(mya_neighbours, y = bbox)
# Combine the reservoir sf object with information about clusters
res_bgmm_feat_co2 <- 
    merge(res_centroids, bgmm_feat_co2, by.x = "name", by.y = "Reservoir", all = TRUE)
res_bgmm_feat_ch4 <- 
    merge(res_centroids, bgmm_feat_ch4, by.x = "name", by.y = "Reservoir", all = TRUE)

## Create voronoi patches with st_voronoi

In [ ]:
pts_vor_feat_co2 <- res_bgmm_feat_co2 %>% 
    distinct(center_lon, center_lat, .keep_all = TRUE) %>%
    st_union() %>%    #merge points into a MULTIPOINT
    st_voronoi() %>%              #calculate voronoi polygons
    st_cast() %>%                 #next three lines return it to a useable list of polygons
    data.frame(geometry = .) %>%  
    st_sf(.) %>%
    st_join(., res_bgmm_feat_co2)               #merge with clusters and other variables

pts_vor_feat_ch4 <- res_bgmm_feat_ch4 %>% 
    distinct(center_lon, center_lat, .keep_all = TRUE) %>%
    st_union() %>%    #merge points into a MULTIPOINT
    st_voronoi() %>%              #calculate voronoi polygons
    st_cast() %>%                 #next three lines return it to a useable list of polygons
    data.frame(geometry = .) %>%  
    st_sf(.) %>%
    st_join(., res_bgmm_feat_ch4)               #merge with clusters and other variables

In [ ]:
#head(res_k_means_feat)
res_filt_co2 <- res_bgmm_feat_co2 |> drop_na(tot_em)
max(res_filt_co2$co2_net)
min(res_filt_co2$co2_net)

In [ ]:
res_filt_ch4 <- res_bgmm_feat_ch4 |> drop_na(tot_em)
max(res_filt_ch4$ch4_net)
min(res_filt_ch4$ch4_net)

## CLUSTERS IN THE FEATURES SPACE

## TODO: Save this plot template for plotting emission intensities - use ROR HP too

In [ ]:
mya_gg <- ggplot(lims_method = "geometry_bbox")
mya_gg <- mya_gg +
    geom_sf(fill = "lightblue3", alpha=0.9) +
    geom_sf(data=mya_outline, fill='white', col="black", alpha=1, size = 5.5)
if (plot_map_with_elevation == TRUE)
{
    mya_gg <- mya_gg + geom_raster(data=mya_dem.df, aes(x=x, y=y,fill = elevation))
}
mya_gg <- mya_gg +
    #geom_sf(fill = "lightblue3", alpha=0.9) +
    #geom_sf(data=mya_outline, fill='white', col="black", alpha=1, size = 5.5) +

    # Temporarily disable elevation
    #geom_raster(data=mya_dem.df, aes(x=x, y=y,fill = elevation)) +



    #scale_fill_gradient(low = "yellow", high = "red", limits = c(0,6000)) +
      scale_fill_continuous_divergingx(
        rev = TRUE,
        palette = "Earth",
        mid = 1500,
        limits = c(0, 6000),
        breaks = 1500*(0:4),
        #labels = c("0% ", "25%", "50%", "75%", " 100%"),
        name = "elevation, m.a.s.l.",
        guide = guide_colorbar(
          #direction = "horizontal",
          #label.position = "bottom",
          #title.position = "top",
          ticks = FALSE
          #barwidth = grid::unit(0.2, "in"),
          #barheight = grid::unit(3, "in"))
      )) +
    geom_sf(data=mya_outside,fill="aliceblue", alpha=1, size=0, color=NA) +
    geom_sf(data=mya_neighbours_crop, fill="lightgray", col="black", alpha=0.6, linewidth=0.25) +
    geom_sf_label(
        data=mya_neighbours_crop %>% filter(COUNTRY != "Bangladesh"), 
        aes(label = COUNTRY), size=3, fill = "white") + 
    geom_sf(data=mya_rivers, col="lightblue4") +
    geom_sf(data = res_bgmm_feat_co2 %>% drop_na() %>% arrange(desc(tot_em)),
          aes(size = tot_em, shape=factor(reservoir_type)), col = "black", fill="white") +
    scale_size(
        limits = c(0,15000),breaks = c(20, 100, 1000, 4000, 10000, 15000),
        name = paste0("Emission\nintensity"), range = c(0.01,14.0)) +
    scale_shape_manual(
        name = paste0("Reservoir\nType"),
        values=c(21,22,23)) + #name = 'Reservoir\nType'
    geom_tile(show.legend = TRUE) +
    # Remove labels
    xlab(NULL) + ylab(NULL) +
    # Map theme
    theme_minimal() +
    theme(
        panel.grid.major = element_line(colour = gray(0.5), linetype = "dashed", size = 0.5), 
        panel.ontop = TRUE,
        panel.background = element_rect(fill = NA), 
        panel.border = element_rect(fill = NA),
        #legend.title = element_markdown(),
        legend.direction = "vertical", 
        #legend.position = "bottom",
        plot.title = element_text(hjust = 0)) +
    # Add annotations for the coastal waters
    annotate(geom = "text", x = 820000, y = 1500000, label = "Andaman\nSea", 
     fontface = "italic", color = "grey22", size = 3) +
    annotate(geom = "text", x = 505000, y = 1950000, label = "Bay\nof\nBengal", 
     fontface = "italic", color = "grey22", size = 3) +
    # Add NORTH ARROW
    ggspatial::annotation_scale(
        location = "br",
        bar_cols = c("grey60", "white"),
        text_family = "Avenir Next Condensed") +
    ggspatial::annotation_north_arrow(
        location = "tl", which_north = "true",
        pad_x = unit(0.12, "in"), pad_y = unit(0.12, "in"),
        style = ggspatial::north_arrow_fancy_orienteering(
          fill = c("grey40", "white"),
          line_col = "grey20",
          text_family = "Avenir Next Condensed"))+
    facet_wrap(~reservoir_type) +
    xlim(bbox["xmin"]+Dx, bbox["xmax"]-Dx) + ylim(bbox["ymin"]+Dy, bbox["ymax"]-Dy) +
    coord_sf(crs = mya_epsg_code)

figure_output_folder <- file.path("figures", "maps")

if (!dir.exists(figure_output_folder)) {
  # If not, create the folder and its parent folders recursively
  dir.create(figure_output_folder, recursive = TRUE)
  cat(paste("The folder at", figure_output_folder, "has been created.\n"))
} else {
  cat(paste("The folder at", figure_output_folder, "already exists. NOT CREATING.\n"))
}
ggsave(file.path(figure_output_folder, "elevation.png"), width = 27, height = 20, units = "cm")

In [ ]:
mya_gg <- ggplot(lims_method = "geometry_bbox")
mya_gg +
    geom_sf(fill = "lightblue3", alpha=0.9) +
    geom_sf(data = st_as_sfc(bbox), fill = NA, color = "red", size = 1) +
    geom_sf(data=mya_outline, fill='white', col="black", alpha=1, size = 5.5) +
    geom_sf(data=pts_vor_feat_ch4, aes(fill=factor(cluster)), alpha=0.6, size=0, color=NA) +
    scale_shape_manual(
        name = paste0("Reservoir\nType"),
        values=c(21,22,23)) + #name = 'Reservoir\nType'
    geom_sf(data=mya_outside,fill="aliceblue", alpha=1, size=0, color="black") +
    geom_sf(data=mya_neighbours_crop, fill="lightgray", col="black", alpha=0.5, size=3) +
    geom_sf_label(
        data=mya_neighbours_crop %>% filter(COUNTRY != "Bangladesh"), 
        aes(label = COUNTRY), fill = "white") + 

    geom_sf(data=mya_rivers, col="lightblue3") +

    geom_sf(data = res_bgmm_feat_ch4 %>% drop_na() %>% arrange(desc(ch4_net)),
          aes(size = ch4_net, shape=factor(reservoir_type)), fill = "white",
          col = "black", alpha=0.6) +
    guides(
        colour = guide_legend(override.aes = list(size=15)),
        shape = guide_legend(override.aes = list(size=4))) +
    #guides(cluster = 'legend', name = "Cluster") +
    
    geom_sf(data = st_as_sfc(bbox), fill = NA, color = "red", size = 1) +
    coord_sf(crs = mya_epsg_code) + 
    labs(title = "Categories of reservoirs with respect to \nmethane emission drivers") +
    # geom_text(aes(label=cluster)) +
    scale_fill_manual(
            name = paste0("Category"),
            values = cluster_colors) +
    scale_radius(
        name=TeX(
            r"(\overset{\normalsize{Aerial CH$_4$ emission}}{\overset{\normalsize{gCO$_{2e}$/m$^2$/yr}}})"),
        range = c(1,10.0), 
        limits = c(50,10000),breaks = c(200, 1000, 3000, 6000, 8000)) +
    # Remove labels
    xlab(NULL) + ylab(NULL) +
    # Map theme
    theme_minimal() +
    theme(
        panel.grid.major = element_line(
            colour = gray(0.5), linetype = "dashed", size = 0.5), 
        panel.ontop = TRUE,
        panel.background = element_rect(fill = NA), 
        panel.border = element_rect(fill = NA),
        #legend.title = element_markdown(),
        text = element_text(size = 14),
        axis.text.x = element_text(size=15),
        axis.text.y = element_text(size=15),
        legend.direction = "vertical", 
        legend.key.size = unit(1.3,"line"),
        #legend.position = "bottom",
        plot.title = element_text(hjust = 0)) +
    # Add annotations for the coastal waters
    annotate(geom = "text", x = 820000, y = 1500000, label = "Andaman\nSea", 
     fontface = "italic", color = "grey22", size = 4) +
    annotate(geom = "text", x = 505000, y = 1950000, label = "Bay\nof\nBengal", 
     fontface = "italic", color = "grey22", size = 4) +
    # Add NORTH ARROW
    ggspatial::annotation_scale(
        location = "tl",
        bar_cols = c("grey60", "white"),
        text_family = "Avenir Next Condensed") +
    ggspatial::annotation_north_arrow(
        location = "tl", which_north = "true",
        pad_x = unit(0.12, "in"), pad_y = unit(0.3, "in"),
        style = ggspatial::north_arrow_fancy_orienteering(
          fill = c("grey40", "white"),
          line_col = "grey20",
          text_family = "Avenir Next Condensed"))+
    xlim(
        bbox["xmin"]+Dx, 
        bbox["xmax"]-Dx) + 
    ylim(
        bbox["ymin"]+Dy, 
        bbox["ymax"]-Dy) +
    coord_sf(crs = mya_epsg_code)

lines <- grid.get("grill.gTree", grep=TRUE)
grid.draw(lines$children[-1])

if (!dir.exists(figure_output_folder)) {
  # If not, create the folder and its parent folders recursively
  dir.create(figure_output_folder, recursive = TRUE)
  cat(paste("The folder at", figure_output_folder, "has been created.\n"))
} else {
  cat(paste("The folder at", figure_output_folder, "already exists.\n"))
}

ggsave(file.path(figure_output_folder, "res_clusters_ch4.png"), width = 27, height = 20, units = "cm")
ggsave(file.path(figure_output_folder, "res_clusters_ch4.svg"), width = 27, height = 20, units = "cm")

In [ ]:
mya_gg <- ggplot(lims_method = "geometry_bbox")
mya_gg +
    geom_sf(fill = "lightblue3", alpha=0.9) +
    geom_sf(data = st_as_sfc(bbox), fill = NA, color = "red", size = 1) +
    geom_sf(data=mya_outline, fill='white', col="black", alpha=1, size = 5.5) +
    geom_sf(data=pts_vor_feat_co2, aes(fill=factor(cluster)), alpha=0.6, size=0, color=NA) +
    scale_shape_manual(
        name = paste0("Reservoir\nType"),
        values=c(21,22,23)) + #name = 'Reservoir\nType'
    geom_sf(data=mya_outside,fill="aliceblue", alpha=1, size=0, color="black") +
    geom_sf(data=mya_neighbours_crop, fill="lightgray", col="black", alpha=0.5, size=3) +
    geom_sf_label(
        data=mya_neighbours_crop %>% filter(COUNTRY != "Bangladesh"), 
        aes(label = COUNTRY), fill = "white") + 

    geom_sf(data=mya_rivers, col="lightblue3") +

    geom_sf(data = res_bgmm_feat_co2 %>% drop_na() %>% arrange(desc(co2_net)),
          aes(size = co2_net, shape=factor(reservoir_type)), fill = "white",
          col = "black", alpha=0.6) +
    guides(
        colour = guide_legend(override.aes = list(size=15)),
        shape = guide_legend(override.aes = list(size=4))) +
    #guides(cluster = 'legend', name = "Cluster") +
    
    geom_sf(data = st_as_sfc(bbox), fill = NA, color = "red", size = 1) +
    coord_sf(crs = mya_epsg_code) + 
    labs(title = "Categories of reservoirs with respect to \ncarbon dioxide emission drivers") +
    # geom_text(aes(label=cluster)) +
    scale_fill_manual(
            name = paste0("Category"),
            values = cluster_colors) +
    scale_radius(
        name=TeX(
            r"(\overset{\normalsize{Aerial CO$_2$ emission}}{\overset{\normalsize{gCO$_{2e}$/m$^2$/yr}}})"),
        range = c(0.2,6.0), 
        limits = c(50,700),breaks = c(100, 250, 500, 700)) +
    # Remove labels
    xlab(NULL) + ylab(NULL) +
    # Map theme
    theme_minimal() +
    theme(
        panel.grid.major = element_line(
            colour = gray(0.5), linetype = "dashed", size = 0.5), 
        panel.ontop = TRUE,
        panel.background = element_rect(fill = NA), 
        panel.border = element_rect(fill = NA),
        #legend.title = element_markdown(),
        text = element_text(size = 14),
        axis.text.x = element_text(size=15),
        axis.text.y = element_text(size=15),
        legend.direction = "vertical", 
        legend.key.size = unit(1.3,"line"),
        #legend.position = "bottom",
        plot.title = element_text(hjust = 0)) +
    # Add annotations for the coastal waters
    annotate(geom = "text", x = 820000, y = 1500000, label = "Andaman\nSea", 
     fontface = "italic", color = "grey22", size = 4) +
    annotate(geom = "text", x = 505000, y = 1950000, label = "Bay\nof\nBengal", 
     fontface = "italic", color = "grey22", size = 4) +
    # Add NORTH ARROW
    ggspatial::annotation_scale(
        location = "tl",
        bar_cols = c("grey60", "white"),
        text_family = "Avenir Next Condensed") +
    ggspatial::annotation_north_arrow(
        location = "tl", which_north = "true",
        pad_x = unit(0.12, "in"), pad_y = unit(0.3, "in"),
        style = ggspatial::north_arrow_fancy_orienteering(
          fill = c("grey40", "white"),
          line_col = "grey20",
          text_family = "Avenir Next Condensed"))+
    xlim(
        bbox["xmin"]+Dx, 
        bbox["xmax"]-Dx) + 
    ylim(
        bbox["ymin"]+Dy, 
        bbox["ymax"]-Dy) +
    coord_sf(crs = mya_epsg_code)

lines <- grid.get("grill.gTree", grep=TRUE)
grid.draw(lines$children[-1])

if (!dir.exists(figure_output_folder)) {
  # If not, create the folder and its parent folders recursively
  dir.create(figure_output_folder, recursive = TRUE)
  cat(paste("The folder at", figure_output_folder, "has been created.\n"))
} else {
  cat(paste("The folder at", figure_output_folder, "already exists.\n"))
}

ggsave(file.path(figure_output_folder, "res_clusters_co2.png"), width = 27, height = 20, units = "cm")
ggsave(file.path(figure_output_folder, "res_clusters_co2.svg"), width = 27, height = 20, units = "cm")